In [434]:
import numpy as np
import pandas as pd
from pandas.api.types import is_numeric_dtype
import sweetviz as sv
import autoviz as av
from scipy import stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import recall_score
pd.set_option('display.max_rows', 100)

In [2]:
'%matplotlib inline'

'%matplotlib inline'

In [297]:
#Reads Data and does some basic cleaning
df = pd.read_csv('./ClientBehaviourTraining.txt', sep='|')
df = df.drop(['Instalment_Amt_All', 'Current_Balance_All'], axis = 1)
df = df.dropna()

#Q1 = df.quantile(0.05)
#Q3 = df.quantile(0.95)
#IQR = Q3 - Q1

#df = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]

#for index in range(9,81):
#    normalized_df=(df-df.mean())/df.std()
    

df.head()


,ClientIdentifier,Behaviour_Date_Key,BehaviourType,TAB1,TAB5,TAB7,TAB8,TAB9,TAB10,Total_Num_Inflow_L3M,...,Num_Elec,Amt_Elec,Num_Airtime,Amt_Airtime,CW1,CS1,CS2,CS3,CS4,InsufFunds
1,320818,11518,CreditCardTake-up,0.0,0.0,12.0,9.0,8.0,0.0,10.0,...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0000,0.0
2,302372,11518,CreditCardTake-up,0.0,0.0,2.0,2.0,2.0,0.0,16.0,...,0.0,0.0,0.0,0.0,-99.000,3.0,3.0,3.0,0.8275,0.0
3,1027545,11518,CreditCardTake-up,0.0,0.0,3.0,2.0,1.0,0.0,20.0,...,0.0,0.0,1.0,-50.0,4.009,3.0,2.0,4.0,0.3400,1.0
4,1809000,11518,CreditCardTake-up,0.0,0.0,23.0,23.0,23.0,0.0,8.0,...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0000,0.0
5,1518005,11518,VirtualCardTake-up,0.0,0.0,2.0,1.0,1.0,0.0,23.0,...,0.0,0.0,7.0,-194.0,-99.000,0.0,0.0,0.0,0.0000,0.0


In [5]:
df['Total_Amt_Inflow_L3M'].max()

238046.94

In [6]:
#Generates Report
my_report = sv.analyze(normalized_df)
my_report.show_html()

                                             |                                             | [  0%]   00:00 ->…

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [151]:
def IQR_Rejection(data_frame, column_name):
    q1 = data_frame[column_name].quantile(0.25)
    q3 = data_frame[column_name].quantile(0.75)
    iqr = q3 - q1
    fence_low  = q1 - 3*iqr
    fence_high = q3 + 3*iqr
    df_out = data_frame.loc[(data_frame[column_name] > fence_low) & (data_frame[column_name] < fence_high)]
    
    return df_out

In [460]:
#Data cleaning
clean = df
has_outliers = ['Total_Num_Inflow_L3M', 'Total_Amt_Inflow_L3M', 'Total_Num_Outflow_L3M', 'Total_Amt_Outflow_L3M', 'Num_Elec_In_G1_L3M', 'Num_CW_ATM_Out_G1_L3M', 'Num_POS_Out_G1_L3M']
redundant = ['Num_Inflow_G1_L3M','Num_Pmt_In_G1_L3M', 'Num_Prepaid_Out_G1_L3M']#was here
binary = ['Num_MoneySend_In_G1_L3M', 'Num_EDO_Out_G1_L3M', 'Num_Loan_Out_G1_L3M', 'Num_SO_Out_G1_L3M']

clean = clean.drop(['ClientIdentifier', 'Behaviour_Date_Key', 'ALL_Max_Delq_Ever'], axis = 1)

one_hot = pd.get_dummies(clean['Has_Deed'])
one_hot = one_hot.drop('N', axis = 1)
clean = clean.drop('Has_Deed', axis = 1)
clean = clean.join(one_hot)

#for (columnName, columnData) in clean.iteritems():
#    if (columnData.unique().size > 2) & ((columnData == 0).mean() > 0.3):
#        clean[columnName] = np.where(columnData == 0, 0, 1)
        


#clean = clean.loc[:,~clean.apply(lambda x: x.duplicated(),axis=1).all()].copy()
#clean.shape

for (columnName, columnData) in clean.iteritems():
   if (columnData.unique().size > 2 and columnName != 'BehaviourType'):
        clean = IQR_Rejection(clean, col)
        #if columnData.min() > 0:
            #clean[columnName] = np.log(clean[columnName])
        #clean = clean.drop(columnName, axis = 1)
clean.shape

(375215, 76)

In [110]:
my_report = sv.analyze(clean)
my_report.show_html()

                                             |                                             | [  0%]   00:00 ->…

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [376]:
edf = pd.read_csv('./ClientBehaviourEvaluation.txt', sep='|')
edf = edf.drop(['Instalment_Amt_All', 'Current_Balance_All'], axis = 1)
edf = edf.dropna()


In [455]:
eclean = edf
eclean = eclean.drop(['ClientIdentifier', 'Behaviour_Date_Key', 'ALL_Max_Delq_Ever'], axis = 1)

one_hot = pd.get_dummies(eclean['Has_Deed'])
one_hot = one_hot.drop('N', axis = 1)
eclean = eclean.drop('Has_Deed', axis = 1)
eclean = eclean.join(one_hot)

#for (columnName, columnData) in eclean.iteritems():
#    if (columnData.unique().size > 2) & ((columnData == 0).mean() > 0.3):
#        eclean[columnName] = np.where(columnData == 0, 0, 1)
    
    
eclean.shape

(179192, 76)

In [378]:
eclean = eclean.loc[:,~eclean.apply(lambda x: x.duplicated(),axis=1).all()].copy()
eclean.shape

(179192, 56)

In [462]:
behavname = 'CreditCard' #Enter behaviour name here

behav = behavname + 'Take-upOOT'
nobehav = behavname + 'NoTake-upOOT'
eappdf = eclean[eclean['BehaviourType'] == behav]
nodf = eclean[eclean['BehaviourType'] == nobehav]
eappdf = pd.concat([eappdf, nodf])
    
one_hot = pd.get_dummies(eappdf['BehaviourType'])
one_hot = one_hot.drop(nobehav, axis = 1)
eappdf = eappdf.drop('BehaviourType', axis = 1)
eappdf = eappdf.join(one_hot)

eX = eappdf.drop(behav, axis = 1).to_numpy()
ey = eappdf[behav].to_numpy()

scaler = preprocessing.StandardScaler().fit(eX)
eX = scaler.transform(eX)

print(balanced_accuracy_score(ey, clf.predict(eX)))
print(recall_score(ey, clf.predict(eX)))

0.7786489615709199
0.9705263157894737


In [459]:
probs = clf.predict_proba(eX)[:,1]
corr = ey.mean()
bal = y.mean()
term = (corr/bal)*probs
truprobs = term / (term + ((1-corr)/(1-bal))*(1-probs))
classi = np.round(truprobs)
print(balanced_accuracy_score(ey, classi))
print(recall_score(ey, classi))

0.7799450425045797
0.6326315789473684


In [428]:
ey.mean()

0.04438008035130337

In [461]:
#Generates correlation coefficients to specific behaviour
behavname = 'CreditCard' #Enter behaviour name here
behav = behavname + 'Take-up'
nobehav = behavname + 'NoTake-up'
appdf = clean[clean['BehaviourType'] == behav]#.sample(frac=ey.mean())
nodf = clean[clean['BehaviourType'] == nobehav]
appdf = pd.concat([appdf,nodf])

one_hot = pd.get_dummies(appdf['BehaviourType'])
one_hot = one_hot.drop(nobehav, axis = 1)
appdf = appdf.drop('BehaviourType', axis = 1)
appdf = appdf.join(one_hot)


from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from scipy import stats


train, test = train_test_split(appdf, test_size=0.2)

X = train.drop(behav, axis = 1).to_numpy()
y = train[behav].to_numpy()

tX = test.drop(behav, axis = 1).to_numpy()
ty = test[behav].to_numpy()

scaler = preprocessing.StandardScaler().fit(X)
#X = preprocessing.normalize(X, norm='l1')
#tX = preprocessing.normalize(tX, norm='l1')
X = scaler.transform(X)
tX = scaler.transform(tX)

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=10000, class_weight = 'balanced', solver='saga').fit(X, y)
print(balanced_accuracy_score(ty, clf.predict(tX)))
print(recall_score(ey, clf.predict(eX)))

0.8265060849366258
0.9705263157894737


In [441]:
appdf.shape

(21127, 56)

In [421]:
from sklearn.neighbors import KNeighborsClassifier
KNeighborsClassifier(n_neighbors=11)
clf = clf.fit(X, y)
print(balanced_accuracy_score(ty, clf.predict(tX)))

0.8344959444284421


0.7442365649856402


In [369]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(activation = 'relu', solver='adam', hidden_layer_sizes=(100), random_state=1, max_iter=10000)
clf = clf.fit(X, y)
a = (clf.predict(tX) == ty).mean()
false_pos = np.sum((clf.predict(tX) == 1) & (ty == 0))
b = false_neg / np.sum(ty == 0)
false_neg = np.sum((clf.predict(tX) == 0) & (ty == 1))
c = false_neg / np.sum(ty == 1)
(a, b, c)

(0.9213456526889363, 0.05155155155155155, 0.8933333333333333)

In [ ]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(ty, clf.predict(tX))

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=10000, class_weight={0.95,0.05}).fit(X, y)
a = (clf.predict(tX) == ty).mean()
false_neg = np.sum((clf.predict(tX) == 0) & (ty == 1))
b = false_neg / np.sum(ey == 1)
(a, b)